# N-Body Simulation

## A 2D n-body simulation that calculates the forces from computing the gradient of the potential.  


In [1]:
# Let's import the necessary libraries

import os
import imageio
import numba as nb
import numpy as np
from scipy import fft
from IPython.display import Image
from matplotlib import pyplot as plt
plt.ion()

Calculating the forces on every pair of particles is tedious and the work scale goes as $n^2$. Let us instead look at the potential from a distribution of particles. If the potential of one particle is $P(r)$, then the potential from the field is given by,

$$ P_{total}(r^{'}) = \int P(r - r^{'}) * \rho(r) d^3r$$

This is nothing but the potential from a single particle convolved with the density field.

### The potential is found by,

   . Gridding the particle positions into densities
   
   . Convolving these FFT of densities with the FFT softened potential from a single particle
   
   . The acceleration/force is found by taking the gradient of the potential. 
   
### We use a leapfrog solver with fixed timestep to update the position and velocity.

$$ \frac{f(t + dt, x) - f(t - dt, x)}{2 * dt} = -v \frac{f(t, x + dx) - f(t, x - dx)}{2 * dx} $$


### Concept of Softening

When, $r$, the distance between the masses goes to $0$, the force goes to $\infty$. To avoid this, we consider the particles to be fuzzy balls rather than point masses. Thus the force drops as they get close to each other! Thus,

$$ \frac{F}{m_1} = a = \frac{G m_2}{(r^2 + \epsilon^2)^{3/2}} * R $$

In [11]:
# Get the Potential from convolving the density with softened potential

# Get the force from the gradient of the potential

@nb.njit(parallel=True)
def get_grad(xy,pot,grad):   # Gives the Force from the gradient of the potential
    n=pot.shape[0]
    for i in nb.prange(xy.shape[0]):
        if xy[i,0]<0:
            ix0=n-1
            ix1=0
            fx=xy[i,0]+1
        else:
            ix0=int(xy[i,0])
            ix1=ix0+1
            fx=xy[i,0]-ix0
            if ix1==n:
                ix1=0
        if xy[i,1]<0:
            iy0=n-1
            iy1=0
            fy=xy[i,1]+1
        else:
            iy0=int(xy[i,1])
            iy1=iy0+1
            fy=xy[i,1]-iy0
            if iy1==n:
                iy1=0
       
        grad[i,0]=(pot[ix1,iy1]-pot[ix0,iy1])*fy+(pot[ix1,iy0]-pot[ix0,iy0])*(1-fy)
        grad[i,1]=(pot[ix1,iy1]-pot[ix1,iy0])*fx+(pot[ix0,iy1]-pot[ix0,iy0])*(1-fx)

def soften_potential(n,soft):    # Function to perform 'Softening the Potential'
    x=np.fft.fftfreq(n)*n
    rsqr=np.outer(np.ones(n),x**2)
    rsqr=rsqr+rsqr.T
    rsqr[rsqr<soft**2]=soft**2
    kernel=rsqr**-0.5
    return kernel

@nb.njit
def get_density(xy,mat,m):   # Function to grid particle positions into density
    nx=xy.shape[0]
    for i in range(nx):
        ix=int(xy[i,0]+0.5)
        iy=int(xy[i,1]+0.5)
        if m[i]>0: 
            mat[ix,iy]=mat[ix,iy]+m[i]

class Particles:
    
    def __init__(self,xy,npar,v,n=1000,soft=0.01):
        self.npar=npar  # Number of Particles
        self.ngrid=n    # Density Grid
        self.xy=xy      # Position
        self.v=v        # Velocity
        self.f=np.zeros([npar,2])     # Force or Acceleration
        self.m=np.zeros(npar)          # Mass
        self.grad=np.empty([npar,2])  # Gradient of the Potential
        self.soft=soft                # Softening Parameter

        self.kernel=[]
        self.kernelft=[]

        self.rho=np.zeros([self.ngrid,self.ngrid])  # Density
        self.potential=np.empty([self.ngrid,self.ngrid])  # Potential Energy
        
    def get_kernel(self):  # Get the softened potential to be convolved with the desnity
        self.kernel = soften_potential(self.ngrid,self.soft)
        self.kernelft = fft.rfft2(self.kernel)

    def get_potential(self):  # Get the potential
        get_density(self.xy,self.rho,self.m)    # Get the Density by gridding the particle positions
        rhoft = fft.rfft2(self.rho)
        self.potential = fft.irfft2(rhoft*self.kernelft,[self.ngrid,self.ngrid]) # Convolving density with softened potential
    
    def get_forces(self):  # Get the force or acceleration
        get_grad(self.xy,self.potential,self.grad)  # Gradient of the potential
        self.f[:]=self.grad
        
    def update(self,dt):  # Update the position and velocity of particle using the leap frog scheme
        self.xy[:]=self.xy[:]+dt*self.v
        self.get_potential()
        self.get_forces()
        self.v[:]=self.v[:]+self.f*dt

### Part 1 -  A single particle starting at rest remains motionless

In [3]:
npar = 1  # Number of particles
xy = np.zeros([npar,2])
v = np.zeros([npar,2])  # Initial Velocity set to zero
dt = 0.0001
nstep = 100 # Number of Iterations
parts = Particles(xy,npar,v)
parts.get_kernel()
    
energy = open('Part 1 - Energy.txt','w')  # File to record the kinetic and potential energies of the particle
filenames = []
for i in range(nstep):
    plt.clf()
    plt.plot(parts.xy[:,0],parts.xy[:,1],'.',label=f'Iteration Number is {i}')
    plt.legend()
    plt.xlim(-2,2)
    plt.ylim(-2,2)
    filename = f'{i}.png'
    filenames.append(filename)
    
    plt.savefig(filename)   # Save the frame
    plt.close()
    
    parts.update(dt)
    kinetic = np.sum(parts.v**2)
    potential = np.sum(parts.rho*parts.potential)
    energy.write('   Kintetic: '+ repr(kinetic) +'        Potential: '+ repr(potential) + 
                 '       total: '+ repr(kinetic-(potential/2))+ '\n')
energy.close()
    
# Build GIF
with imageio.get_writer('Single Particle at Rest.gif', mode='I') as writer:  # This GIF can be found in the folder
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove Files
for filename in set(filenames):
    os.remove(filename) 

From the $\textbf{'Single Particle at Rest'}$ GIF found in this folder, we see that the particle is rest throughout the simulation as the velocity is set to zero (Particle starting at rest) and there are no forces acting on the particle

### Part 2 - A pair of particles placed in a circular orbit continue to orbit each other

In [10]:
npar = 2   # Number of particles
xy = np.zeros([npar,2])
xy[1, 1] = 0.5
v = np.zeros([npar,2])
# To set the particles in circular motion, the velocity of 1st particle should be equal but opposite to the 2nd particle
v[0,0] = 2
v[0,1] = 2
v[1,0] = -2
v[1,1] = -2
dt = 3e-4
nstep = 500 # Number of Iterations
parts = Particles(xy,npar,v)
parts.get_kernel()

energy = open('Part 2 Energy.txt','w')  # File to record the Kinetic and Potetial Energies of the Particle
filenames = []
for i in range(nstep):
    plt.clf()
    plt.plot(parts.xy[:,0],parts.xy[:,1],'.',label=f'Iteration Number is {i}')
    plt.legend()
    plt.xlim(-1,1)
    plt.ylim(-1,1)
    filename = f'{i}.png'
    filenames.append(filename)
    
    plt.savefig(filename)   # Save the frame
    plt.close()
    
    parts.update(dt)
    kinetic = np.sum(parts.v**2)
    potential = np.sum(parts.rho*parts.potential)
    energy.write('   Kintetic: '+ repr(kinetic) +'        Potential: '+ repr(potential) + 
                 '       total: '+ repr(kinetic-(potential/2))+ '\n')
energy.close()
    
# Build GIF
with imageio.get_writer('Two Particle.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)  

From the  $\textbf{'Two Particles in circle'}$  GIF found in this folder, we see that the particles orbit each other for a considerable amount of time and later start drifting apart. 

### Part 3 - Periodic and Non-Periodic Boundary Conditions

In [4]:
@nb.njit(parallel=True)   # Gives the Force from the gradient of the potential
def get_grad(xy,pot,grad):
    n=pot.shape[0]
    for i in nb.prange(xy.shape[0]):
        if xy[i,0]<0:
            ix0=n-1
            ix1=0
            fx=xy[i,0]+1
        else:
            ix0=int(xy[i,0])
            ix1=ix0+1
            fx=xy[i,0]-ix0
            if ix1==n:
                ix1=0
        if xy[i,1]<0:
            iy0=n-1
            iy1=0
            fy=xy[i,1]+1
        else:
            iy0=int(xy[i,1])
            iy1=iy0+1
            fy=xy[i,1]-iy0
            if iy1==n:
                iy1=0

        grad[i,0]=(pot[ix1,iy1]-pot[ix0,iy1])*fy+(pot[ix1,iy0]-pot[ix0,iy0])*(1-fy)
        grad[i,1]=(pot[ix1,iy1]-pot[ix1,iy0])*fx+(pot[ix0,iy1]-pot[ix0,iy0])*(1-fx)
        

@nb.njit
def get_density(xy,mat,m):  # Function to grid particle positions into density
    nx=xy.shape[0]
    for i in range(nx):
        ix=int(xy[i,0]+0.5)
        iy=int(xy[i,1]+0.5)
        if m[i]>0: 
            mat[ix,iy]=mat[ix,iy]+m[i]
    
def soften_potential(n,soft):   # Function to perform 'Softening the Potential'
    x=np.fft.fftfreq(n)*n
    rsqr=np.outer(np.ones(n),x**2)
    rsqr=rsqr+rsqr.T
    rsqr[rsqr<soft**2]=soft**2
    kernel=rsqr**-0.5
    return kernel

class Particles:
    
    def __init__(self,xy,npar,v,n=100,soft=0.1,periodic=True):
        self.npar=npar  # Number of Particles
        self.ngrid=n    # Density Grid
        self.xy=xy      # Position
        self.v=v        # Velocity
        self.f=np.zeros([npar,2])     # Force or Acceleration
        self.m=np.ones(npar)          # Mass
        self.grad=np.empty([npar,2])  # Gradient of the Potential
        self.soft=soft                # Softening Parameter

        self.kernel=[]
        self.kernelft=[]
        self.periodic=periodic
        
        self.rho=np.zeros([self.ngrid,self.ngrid])  # Density
        self.potential=np.empty([self.ngrid,self.ngrid])  # Potential Energy

    def get_kernel(self):   # Get the softened potential to be convolved with the desnity
        self.kernel = soften_potential(self.ngrid,self.soft)   
        self.kernelft = fft.rfft2(self.kernel)
        
    def get_potential(self):  # Get the potential   
        get_density(self.xy,self.rho,self.m)   # Get the Density by gridding the particle positions
        rhoft = fft.rfft2(self.rho)
        self.potential=fft.irfft2(rhoft*self.kernelft,[self.ngrid,self.ngrid])  # Convolving density with softened potential

    def get_forces(self): # Get the force or acceleration
        get_grad(self.xy,self.potential,self.grad)  # Gradient of the Potential
        self.f[:]=self.grad
        
    def update(self,dt):   # Update the position and velocity of particle using the leap frog scheme
        self.xy[:]=self.xy[:]+dt*self.v
        self.get_potential()
        self.get_forces()
        self.v[:]=self.v[:]+self.f*dt
        
        # Periodic Condition
        
        # When a particle passes the boundaries, the location of the particle is changed to the other side of the boundary. 

        if self.periodic is True:
            for i in range(self.npar):
                if self.xy[i, 0]<-3:
                    self.xy[i, 0]= 3-(-3-self.xy[i,0])

                if self.xy[i, 1]<-3 :
                    self.xy[i, 1]= 3-(-3-self.xy[i,1])

                if self.xy[i, 0]>3:
                    self.xy[i, 0]= -3 + (self.xy[i,0]-3)

                if self.xy[i, 1]>3:
                    self.xy[i, 1]= -3 + (self.xy[i,1]-3)
                    
        # Non - Periodic Condition
        
        # Once the particle passes the boundaries, the sign of the force component vertical to the boundary is changed...
        
        # to make it come back to the domain.

        else:
            for i in range(self.npar):
                if self.xy[i, 0]<-3:
                    self.xy[i, 0]= -3
                    self.v[i, 1]= - self.v[i, 1]

                if self.xy[i, 0]>3:
                    self.xy[i, 0]= 3
                    self.v[i, 1]= - self.v[i, 1]

                if self.xy[i, 1]<-3:
                    self.xy[i, 1]= -3
                    self.v[i, 0]= - self.v[i, 0]

                if self.xy[i, 1]>3:
                    self.xy[i, 1]= 3
                    self.v[i, 0]= - self.v[i, 0]

In [5]:
# Periodic Boundary Conditions

np.random.seed(8)
npar = 1000  # Number of particles
xy = np.random.randn(npar,2)
v = np.random.randn(npar,2)
dt = 3e-5
nstep = 100 # Number of Iterations
parts = Particles(xy,npar,v,periodic=True)
parts.get_kernel()

energy = open('Part 3(a) - Energy.txt','w')  # File to record the Kinetic and Potetial Energies of the Particle
filenames = []
for i in range(nstep):
    plt.clf()
    plt.plot(parts.xy[:,0],parts.xy[:,1],'.',label=f'Iteration Number is {i}')
    plt.legend()
    plt.xlim([-3,3])
    plt.ylim([-3,3])
    filename = f'{i}.png'
    filenames.append(filename)
    
    plt.savefig(filename)   # Save the frame
    plt.close()
    
    parts.update(dt)
    kinetic = np.sum(parts.v**2)
    potential = np.sum(parts.rho*parts.potential)
    energy.write('   Kintetic: '+ repr(kinetic) +'        Potential: '+ repr(potential) + 
                 '       total: '+ repr(kinetic-(potential/2))+ '\n')
energy.close()
    
# Build GIF
with imageio.get_writer('Leapfrog Periodic.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)  

In [7]:
# Non - Periodic Boundary Conditions

np.random.seed(8)
npar = 1000   # Number of particles
xy = np.random.randn(npar,2)
v = np.random.randn(npar,2)
dt = 3e-5
nstep = 100 # Number of Iterations
parts = Particles(xy,npar,v,periodic=False)
parts.get_kernel()

energy = open('Part 3(b) - Energy.txt','w')  # File to record the Kinetic and Potetial Energies of the Particle
filenames = []
for i in range(nstep):
    plt.clf()
    plt.plot(parts.xy[:,0],parts.xy[:,1],'.',label=f'Iteration Number is {i}')
    plt.legend()
    plt.xlim([-3,3])
    plt.ylim([-3,3])
    filename = f'{i}.png'
    filenames.append(filename)
    
    plt.savefig(filename)   # Save the frame
    plt.close()
    
    parts.update(dt)
    kinetic = np.sum(parts.v**2)
    potential = np.sum(parts.rho*parts.potential)
    energy.write('   Kintetic: '+ repr(kinetic) +'        Potential: '+ repr(potential) + 
                 '       total: '+ repr(kinetic-(potential/2))+ '\n')
energy.close()
    
# Build GIF
with imageio.get_writer('Leapfrog Non-Periodic.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)  

We see that as time evolves, the gravitational attractive force pulls the particles towards the center!

### Part 4 - RK4 Integrator

Ranga - Kutta Method is used to solve ODEs, $ \frac{dy}{dx} = f(x,y) $

$$ k_1 = h f(x,y) $$

$$ k_2 = h f(x + h/2, y + k_1/2) $$

$$ k_3 = h f(x + h/2, y + k_2/2) $$

$$ k_4 = h f(x + h, y + k_3) $$

Then, $$ y(x + h) = y(x) + \frac{k_1 + 2 k_2 + 2 k_3 + k_4}{6} $$

Where, $h$ is the step size

In [14]:
@nb.njit(parallel=True)   # Gives the Force from the gradient of the potential
def get_grad(xy,pot,grad):
    n=pot.shape[0]
    for i in nb.prange(xy.shape[0]):
        if xy[i,0]<0:
            ix0=n-1
            ix1=0
            fx=xy[i,0]+1
        else:
            ix0=int(xy[i,0])
            ix1=ix0+1
            fx=xy[i,0]-ix0
            if ix1==n:
                ix1=0
        if xy[i,1]<0:
            iy0=n-1
            iy1=0
            fy=xy[i,1]+1
        else:
            iy0=int(xy[i,1])
            iy1=iy0+1
            fy=xy[i,1]-iy0
            if iy1==n:
                iy1=0

        grad[i,0]=(pot[ix1,iy1]-pot[ix0,iy1])*fy+(pot[ix1,iy0]-pot[ix0,iy0])*(1-fy)
        grad[i,1]=(pot[ix1,iy1]-pot[ix1,iy0])*fx+(pot[ix0,iy1]-pot[ix0,iy0])*(1-fx)
        

@nb.njit
def get_density(xy,mat,m):  # Function to grid particle positions into density
    nx=xy.shape[0]
    for i in range(nx):
        ix=int(xy[i,0]+0.5)
        iy=int(xy[i,1]+0.5)
        if m[i]>0: 
            mat[ix,iy]=mat[ix,iy]+m[i]
    
def soften_potential(n,soft):   # Function to perform 'Softening the Potential'
    x=np.fft.fftfreq(n)*n
    rsqr=np.outer(np.ones(n),x**2)
    rsqr=rsqr+rsqr.T
    rsqr[rsqr<soft**2]=soft**2
    kernel=rsqr**-0.5
    return kernel

    
class Particles:
    
    def __init__(self,xy,npar,v,n=1000,soft=0.01,periodic=True):
        self.npar=npar  # Number of Particles
        self.ngrid=n    # Density Grid
        self.xy=xy      # Position
        self.v=v        # Velocity
        self.f=np.zeros([npar,2])     # Force or Acceleration
        self.m=np.zeros(npar)          # Mass
        self.grad=np.empty([npar,2])  # Gradient of the Potential
        self.soft=soft                # Softening Parameter

        self.kernel=[]
        self.kernelft=[]
        self.periodic=periodic
        
        self.rho=np.zeros([self.ngrid,self.ngrid])  # Density
        self.potential=np.empty([self.ngrid,self.ngrid])  # Potential Energy

    def get_kernel(self):   # Get the softened potential to be convolved with the desnity
        self.kernel = soften_potential(self.ngrid,self.soft)   
        self.kernelft = fft.rfft2(self.kernel)
        
    def get_potential(self):  # Get the potential   
        get_density(self.xy,self.rho,self.m)   # Get the Density by gridding the particle positions
        rhoft = fft.rfft2(self.rho)
        self.potential=fft.irfft2(rhoft*self.kernelft,[self.ngrid,self.ngrid])  # Convolving density with softened potential

    def get_forces(self): # Get the force or acceleration
        self.get_potential()
        get_grad(self.xy,self.potential,self.grad)  # Gradient of the Potential
        self.f[:]=self.grad
        return self.f
        
    def get_derivs(self,xx):
        n = xx.shape[0]//2
        x = xx[:n,:]
        v = xx[n:,:]
        f = self.get_forces()
        return np.vstack([v,f])
        
    def rk4_update(self,dt):  # Update the position and velocity of particle using the RK4 scheme
        xx = np.vstack([self.xy,self.v])
        k1 = self.get_derivs(xx)
        k2 = self.get_derivs(xx+k1*dt/2)
        k3 = self.get_derivs(xx+k2*dt/2)
        k4 = self.get_derivs(xx+k3*dt)

        total = (k1+2*k2+2*k3+k4)/6
        self.xy = self.xy + total[:self.npar,:]*dt
        self.v = self.v + total[self.npar:,:]*dt
        
        # Periodic Condition
        
        # When a particle passes the boundaries, the location of the particle is changed to the other side of the boundary. 

        if self.periodic is True:
            for i in range(self.npar):
                if self.xy[i, 0]<-3:
                    self.xy[i, 0]= 3-(-3-self.xy[i,0])

                if self.xy[i, 1]<-3 :
                    self.xy[i, 1]= 3-(-3-self.xy[i,1])

                if self.xy[i, 0]>3:
                    self.xy[i, 0]= -3 + (self.xy[i,0]-3)

                if self.xy[i, 1]>3:
                    self.xy[i, 1]= -3 + (self.xy[i,1]-3)
                    
        # Non - Periodic Condition
        
        # Once the particle passes the boundaries, the sign of the force component vertical to the boundary is changed...
        
        # to make it come back to the domain.

        else:
            for i in range(self.npar):
                if self.xy[i, 0]<-3:
                    self.xy[i, 0]= -3
                    self.v[i, 1]= - self.v[i, 1]

                if self.xy[i, 0]>3:
                    self.xy[i, 0]= 3
                    self.v[i, 1]= - self.v[i, 1]

                if self.xy[i, 1]<-3:
                    self.xy[i, 1]= -3
                    self.v[i, 0]= - self.v[i, 0]

                if self.xy[i, 1]>3:
                    self.xy[i, 1]= 3
                    self.v[i, 0]= - self.v[i, 0]

In [15]:
# Periodic Boundary Conditions

np.random.seed(8)
npar = 1000   # Number of particles
xy = np.random.randn(npar,2)
v = np.random.randn(npar,2)
dt = 3e-4
nstep = 300 # Number of Iterations
parts = Particles(xy,npar,v,periodic=True)
parts.get_kernel()

energy = open('Part 4(a) - Energy.txt','w')  # File to record the Kinetic and Potetial Energies of the Particle
filenames = []
for i in range(nstep):
    plt.clf()
    plt.plot(parts.xy[:,0],parts.xy[:,1],'.',label=f'Iteration Number is {i}')
    plt.legend()
    plt.xlim([-3,3])
    plt.ylim([-3,3])
    filename = f'{i}.png'
    filenames.append(filename)
    
    plt.savefig(filename)   # Save the frame
    plt.close()
    
    parts.rk4_update(dt)
    kinetic = np.sum(parts.v**2)
    potential = np.sum(parts.rho*parts.potential)
    energy.write('   Kintetic: '+ repr(kinetic) +'        Potential: '+ repr(potential) + 
                 '       total: '+ repr(kinetic-(potential/2))+ '\n')
energy.close()
    
# Build GIF
with imageio.get_writer('Periodic.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)  

In [46]:
# Non - Periodic Boundary Conditions

np.random.seed(8)
npar = 1000   # Number of particles
xy = np.random.randn(npar,2)
v = np.random.randn(npar,2)
dt = 3e-4
nstep = 300 # Number of Iterations
parts = Particles(xy,npar,v,periodic=False)
parts.get_kernel()

energy = open('Part 4(b) - Energy.txt','w')  # File to record the Kinetic and Potetial Energies of the Particle
filenames = []
for i in range(nstep):
    plt.clf()
    plt.plot(parts.xy[:,0],parts.xy[:,1],'.',label=f'Iteration Number is {i}')
    plt.legend()
    plt.xlim([-3,3])
    plt.ylim([-3,3])
    filename = f'{i}.png'
    filenames.append(filename)
    
    plt.savefig(filename)   # Save the frame
    plt.close()
    
    parts.rk4_update(dt)
    kinetic = np.sum(parts.v**2)
    potential = np.sum(parts.rho*parts.potential)
    energy.write('   Kintetic: '+ repr(kinetic) +'        Potential: '+ repr(potential) + 
                 '       total: '+ repr(kinetic-(potential/2))+ '\n')
energy.close()
    
# Build GIF
with imageio.get_writer('Non-Periodic.gif', mode='I') as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Remove files
for filename in set(filenames):
    os.remove(filename)  

Since we assume that the work is dominated by the calls to get forces, we see that leapfrog scheme preserves energy better than RK4 Integrator as the work for Leapfrog goes as the number of steps (nstep), while the work for RK4, each step calls the force 4 times!